# Medicare Part D Drugs Deduplication
## Authors: 
    1. Lam Ho
    2. Jonah Breslow
    3. Jeffrey Kagan
## Purpose:
The purpose of this notebook is to internally deduplicate the drug data from the Medicare Part D Drug data. The output will be used to match with other data that contain drug node information. For this procedure, we utilized the Dedupe.io python implementation.

### Importing modules

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
This code demonstrates how to use dedupe with a comma separated values
(CSV) file. All operations are performed in memory, so will run very
quickly on datasets up to ~10,000 rows.

We start with a CSV file containing our messy data. In this example,
it is listings of early childhood education centers in Chicago
compiled from several different sources.

The output will be a CSV with our clustered results.

For larger datasets, see our [mysql_example](mysql_example.html)
"""

import os
import csv
import re
import logging
import optparse
import pandas as pd
import pickle

import dedupe
from unidecode import unidecode

### Running the Dedupe procedure

In [2]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column


def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['MedD_drug_id'])
            data_d[row_id] = dict(clean_row)

    return data_d

In [3]:
# ## Setup
input_file = 'Data/Outputs_Cleanup/Part_d/drug_information.csv'
output_file = 'Data/Outputs_Cleanup/Part_d/dedupe_output/MedicareD_drug_deduplicated.csv'
settings_file = 'Data/Outputs_Cleanup/Part_d/dedupe_output/csv_example_learned_settings'
training_file = 'Data/Outputs_Cleanup/Part_d/dedupe_output/csv_example_training.json'

## Parameters

In [4]:
retrain = False

In [5]:
# Parameters
retrain = False


In [6]:
if retrain == True:
    try:
        os.remove(settings_file)
    except:
        print('Your settings file appears to not have existed.')

#input_file = 'csv_example_messy_input.csv'


print('importing data ...')
data_d = readData(input_file)

# If a settings file already exists, we'll just load that and skip training
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    # ## Training

    # Define the fields dedupe will pay attention to
    fields = [
        {'field': 'brand_name', 'type': 'String'},
        #{'field': 'Address', 'type': 'String'},
        #{'field': 'Zip', 'type': 'Exact', 'has missing': True},
        #{'field': 'Phone', 'type': 'String', 'has missing': True},
        ]

    # Create a new deduper object and pass our data model to it.
    deduper = dedupe.Dedupe(fields)

    # If we have training data saved from a previous run of dedupe,
    # look for it and load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.prepare_training(data_d, f)
    else:
        deduper.prepare_training(data_d)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as duplicates
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(deduper)

    # Using the examples we just labeled, train the deduper and learn
    # blocking predicates
    deduper.train()

    # When finished, save our training to disk
    with open(training_file, 'w') as tf:
        deduper.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        deduper.write_settings(sf)

# ## Clustering

# `partition` will return sets of records that dedupe
# believes are all referring to the same entity.

print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

with open(output_file, 'w') as f_output, open(input_file) as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['MedD_drug_id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

INFO:dedupe.api:Predicate set:


INFO:dedupe.api:(TfidfNGramCanopyPredicate: (0.8, brand_name), SimplePredicate: (sortedAcronym, brand_name), SimplePredicate: (sameSevenCharStartPredicate, brand_name))


INFO:dedupe.api:(SimplePredicate: (sameSevenCharStartPredicate, brand_name), TfidfTextCanopyPredicate: (0.6, brand_name), TfidfNGramCanopyPredicate: (0.4, brand_name))


INFO:dedupe.api:(SimplePredicate: (firstTwoTokensPredicate, brand_name), TfidfTextCanopyPredicate: (0.2, brand_name), LevenshteinCanopyPredicate: (4, brand_name))


INFO:dedupe.api:(SimplePredicate: (commonThreeTokens, brand_name), SimplePredicate: (nearIntegersPredicate, brand_name), TfidfTextCanopyPredicate: (0.8, brand_name))


importing data ...
reading from Data/Outputs_Cleanup/Part_d/dedupe_output/csv_example_learned_settings
clustering...


# duplicate sets 2271


### Post-Processing the Drug Matches

When we cluster multiple drug brand names into the same drug, we create a function that picks the longest brand_name to be the cluster name

In [7]:
def pick_brand(brands):
    pick = None
    pick_len = 0
    for idx, brand in enumerate(brands):
        if idx == 0:
            pick = brand
            pick_len = len(brand)
            continue
        if len(brand) > pick_len:
            pick = brand
            pick_len = len(brand)
    return pick

df = pd.read_csv(output_file)

In [8]:
df_cluster = pd.DataFrame(df.groupby('Cluster ID')['brand_name'].apply(list))
df_cluster['MedD_drug_id'] = df.groupby('Cluster ID')['MedD_drug_id'].apply(list)
# df_cluster

### Exporting Results

exporting a lookup table that has unique Medicare drugs with a reference back to the source data in the `MedD_drug_id` field.

In [9]:
df_cluster['brand_name_picked'] = df_cluster.brand_name.apply(pick_brand)
df_cluster.head(3)

,brand_name,MedD_drug_id,brand_name_picked
Cluster ID,,,
0,"[BUPRENORPHINE, BUPRENORPHINE HCL]","[6, 12251]",BUPRENORPHINE HCL
1,"[DICLOFENAC SODIUM, DICLOFENAC SODIUMMISOPROSTOL]","[11, 78148]",DICLOFENAC SODIUMMISOPROSTOL
2,"[FENTANYL, FENTANYL CITRATE]","[15, 177410]",FENTANYL CITRATE


#### Pickling our table

In [10]:
df_cluster.columns = ['brand_names', 'MedD_drug_id', 'brand_name']
pickle.dump(df_cluster, open("Data/Outputs_Cleanup/Part_d/dedupe_output/MedicareD_Drug_deduplicated_singlebrand.p", "wb" ) )
# df_cluster.to_csv('Data/Outputs_Cleanup/Part_d/dedupe_output/MedicareD_Drug_deduplicated_singlebrand.csv', index=False)